In [1]:
#imports
from LSTM import LSTM
import DataPreparationLSTM
import os
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit

2024-05-10 16:31:26.839999: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
input_data = pd.read_csv('data/SPY_data_5min.csv')
input_data['time'] = pd.to_datetime(input_data['time'])

# Filter trades within the time window from 09:30 to 16:00
input_data = input_data[(input_data['time'].dt.time >= pd.to_datetime('09:30').time()) & 
                 (input_data['time'].dt.time <= pd.to_datetime('16:00').time())]

input_data.drop(['Unnamed: 0','money','open','high','low'], axis=1, inplace=True)
input_data['time'] = pd.to_datetime(input_data['time']).dt.strftime('%H:%M')


In [2]:
learning_rates = [0.001, 0.01]
layer_one =[2,20,40]
layer_two = [2,15,40]
layer_three =[0,4,15]
layer_four = [0,2,10]

In [3]:
configs = {}

for i in range(len(learning_rates)):
    for j in range(len(layer_one)):
        for k in range(len(layer_two)):
            for c in range(len(layer_three)):
                for f in range(len(layer_four)):
                    configs["{}{}{}{}{}".format(i, j, k, c, f)] = [
                        learning_rates[i],
                        layer_one[j],
                        layer_two[k],
                        layer_three[c],
                        layer_four[f],
                    ]

In [3]:
futures = [1,5,20]

In [ ]:
models = {}

for future in futures:
    prepared_data = DataPreparationLSTM(
        df = input_data,
        future = future,
        lag = 20,
        min_max_scaler=True,
        log_transform=True,
        semi_variance=True,
        jump_detect=True,
        period_train= list(
            [
                pd.to_datetie("2006-01-01",format = "%Y-%m-%d"),
                pd.to_datetime("2010-12-31",format = "%Y-%m-%d"),
            ]
        ),
        period_validation = list(
            [
                pd.to_datetime("2011-01-01",format = "%Y-%m-%d"),
                pd.to_datetime("2015-12-31",format = "%Y-%m-%d"),  
            ]
        ),
        period_test= list(
            [
                pd.to_datetime("2016-01-01",format = "%Y-%m-%d"),
                pd.to_datetime("2018-12-31",format = "%Y-%m-%d"),
            ]
        ),
    )

    prepared_data.prepare_all()
    best_config = None
    best_params = None
    for (learning_rate,layer_one,layer_two,layer_three,layer_four) in configs:

        folds = prepared_data.splits
        errors = []
        for (train_data, validate_data) in folds:
            model = LSTM(
                prepared_data.training_set,
                prepared_data.validation_set,
                prepared_data.x_train,
                prepared_data.y_train,
                prepared_data.validation_x,
                prepared_data.validaiton_y,
                epochs= 20,
                learning_rate= learning_rate,
                layer_one=layer_one,
                layer_two= layer_two,
                layer_three=layer_three,
                layer_four = layer_four
        
            )
            model.train_lstm()
            model.make_accuracy_measures()

            errors.append([model.train_accuracy, model.test_accuracy])
        


        model = LSTM(
            prepared_data.training_set,
            prepared_data.val
        )

